In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

In [ ]:
%%html
<style>
.output_wrapper button.btn.btn-default,
.output_wrapper .ui-dialog-titlebar {
  display: none;
}
</style>

In [ ]:
%aiida
%matplotlib notebook
import base64
import pandas as pd
import urllib.parse as urlparse
import ipywidgets as ipw
from IPython.display import display, clear_output
from aiidalab_widgets_base import viewer
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
url = urlparse.urlsplit(jupyter_notebook_url)
uuid = urlparse.parse_qs(url.query)['uuid'][0]
workcalc = load_node(uuid=str(uuid))

molecule_converter = {
    "co2": "CO2",
    "ch4": "CH4",
    "n2" : "N2",
    "h2o": "H2O",
    "h2" : "H2",
    "o2" : "O2",
}

# Detailed Report

## Structure

In [ ]:
display(viewer(workcalc.inputs.structure))

In [ ]:
plot_button = ipw.Button(description="Plot")
text_plot = ipw.Textarea(
    value='',
    placeholder='Pressure [bar] loading [mol/kg]',
    description='Experimental isotherm:',
    disabled=False,
    layout={'width': "40%", 'height': '300px'},
    style = {"description_width": "initial"}
)

def on_apply(_=None):
    to_plot = []
    for line in text_plot.value.splitlines():
        to_plot.append(list(map(float, line.split())))
    res = np.array(to_plot).transpose()
    if res.size > 0:
        line_exp.set_data(res[0], res[1])
    else:
        line_exp.set_data([], [])
    fig.canvas.draw_idle()

plot_button.on_click(on_apply)

## Isotherm and Enthalpy plots

In [ ]:
if 'isotherm' in workcalc.outputs.output_parameters.dict:
    
    # Extracting the necessary data.
    ism = workcalc.outputs.output_parameters['isotherm']
    isotherm = np.array([ism['pressure'], ism['loading_absolute_average'], ism['loading_absolute_dev']]).transpose()
    enthalpy = np.array([ism['enthalpy_of_adsorption_average'], ism['loading_absolute_average'], ism['enthalpy_of_adsorption_dev']]).transpose()

    # Cleaning up the isotherm data.
    to_delete_ism = []
    for i, p in enumerate(isotherm):
        if None in p:
            to_delete_ism.append(i)   
    if to_delete_ism:
        isotherm = np.delete(isotherm, to_delete_ism, axis=0)
    ism_p, ism_load, ism_dev = zip(*isotherm)
    
    # Cleaning up the enthalpy data.
    to_delete_enth = []
    for i, p in enumerate(enthalpy):
        if None in p:
            to_delete_enth.append(i)
    if to_delete_enth:
        enthalpy = np.delete(enthalpy, to_delete_enth, axis=0)
    enth_av, ism_load_enth, enth_dev = zip(*enthalpy)


    # Plots
    
    ## Labels.
    pressure_label = "Pressure [{}]".format(ism['pressure_unit'])
    loading_label = "{} loading [{}]".format(molecule_converter[workcalc.inputs['molecule'].value], ism['loading_absolute_unit'])
    enthalpy_label = "Enthalpy of adsorption [{}]".format(ism['enthalpy_of_adsorption_unit'])
    
    ## Min/max Y values.
    min_y_value = min(np.array(ism_load)-np.array(ism_dev)) * 0.99
    max_y_value = max(np.array(ism_load)+np.array(ism_dev)) * 1.01
    
    ## Plotting the isotherm.
    fig, axs = plt.subplots(1,2, figsize=(9.9, 5))
    line1 = axs[0].errorbar(ism_p, ism_load, yerr=ism_dev, fmt='-or', ecolor='#f58d7f')
    axs[0].set_title("Isotherm")
    axs[0].grid(True)
    axs[0].set_xlabel(pressure_label)
    axs[0].set_ylabel(loading_label)
    axs[0].set_ylim([min_y_value,  max_y_value])
    line_exp, = axs[0].plot([], [], '-ob')

    
    ## Min/max X values:
    min_x_value = min(np.array(enth_av)-np.array(enth_dev))
    max_x_value = max(np.array(enth_av)+np.array(enth_dev))
    if min_x_value > 0:
        min_x_value *= 0.99
    else:
        min_x_value *= 1.01
        
    if max_x_value > 0:
        max_x_value *= 1.01
    else:
        max_x_value *= 0.99
        
    ## The Enthalpy plot.
    line2 = axs[1].errorbar(enth_av, ism_load_enth, xerr=enth_dev, fmt='-or')
    axs[1].set_title("Enthalpy of adsorption [{}]".format(ism['enthalpy_of_adsorption_unit']))
    axs[1].grid(True)
    axs[1].set_xlabel(enthalpy_label)
    axs[1].set_ylabel(loading_label)
    axs[1].set_xlim([min_x_value-2,  max_x_value+2])
    axs[1].set_ylim([min_y_value,  max_y_value])
    

    # CSV
    dataf = pd.DataFrame([(key, ism['loading_absolute_average'][i], ism['enthalpy_of_adsorption_average'][i]) for i, key in enumerate(ism['pressure'])],
                         columns=[pressure_label, loading_label, enthalpy_label])
    table = ipw.HTML('')
    payload = base64.b64encode(dataf.to_csv(index=False).encode()).decode()
    fname = 'data.csv'
    to_add = """Download data in csv format: <a download="{filename}"
    href="data:text/csv;base64,{payload}" target="_blank">{title}</a>"""
    table.value = to_add.format(filename=fname, payload=payload, title=fname)
    display(table)
    box = ipw.VBox(children=[text_plot, plot_button])
    accordion = ipw.Accordion(children=[box], selected_index=None)
    accordion.set_title(0, 'Compare to experiment')
    display(accordion)
else:
    print("No isotherm produced in this calculation.")

## Computed properties

In [ ]:
display(viewer(workcalc.outputs.output_parameters))